# <center><span style="font-size: 50px; color: blue;">ARCHITECTURE DEMO</span></center>

<center><span style="font-size: 25px; color: purple;">This notebook shows how to use our new architecture to perform a simulation</span></center>
<p></p>

<center><span style="font-size: 16px; color: black;">Please make sure you installed our module named <b><i>filament<i/></b> by running the following commands in a terminal:</span></center>
<p></p>

<center><code style="font-size: 16px; color: green;">$ cd PIE_meteo/arch/
$ python setup.py install

# Our module is named filament

In [1]:
from filament.core.grid import Grid
from filament.core.state import State
from filament.core.simulation import Simulation
from filament.test.test_cases import v_stripe_test, bubble_test, gaussian_test

# Simulation parameters

In [2]:
Lx = 2048
Ly = 1024
Nx = 256
Ny = 128


#T =48*3600 Complete simulation is very long
dt = 10
T = 51*dt
Nt = T//dt


dX = Nx//4  # used to shape the initial v-stripe data
dY = Ny//4

nb_state = 2  # number of instants in initial data

# We import the scientific methods (Hi Olivier & Max)

In [3]:
from filament.methods.pseudo_spectral_wind import pseudo_spectral_wind
from filament.methods.wrap_advection_step_3P import wrap_advection_step_3P

Your methods have just been wrapped in such a way it fits our architechture, here is an example of a stupid method. Nothing of scientific, but you can see its signature, and basically understand how we expect a method to interact with data.

In [4]:
def stupid_method(history, verbose, *args, **kwargs):
    last_state = history.state_list[-1]
    
    new_state = State.copy(last_state)
    new_state.t += dt
    
    print(new_state.t) if verbose else None
    
    new_state.vrs['theta_t'][int(new_state.t) % Nx, int(new_state.t) % Ny] = 1 # yes it is stupid
    
    history.append(new_state) # adding the new fresh state
    history.pop(0) # deleting the oldest state

# Simulation parameters and building

In [5]:
methods = [pseudo_spectral_wind,
           wrap_advection_step_3P]

methods_kwargs = [{},
                  {'alpha_method' : 'mix',
                   'order_alpha' : 2, 
                   'F_method' : 'bicubic'}]

output_folder = 'output_test'
save_rate = 1
backup_rate = 10
verbose = 2 # displaying level, usefull to inspect what's going wrong

initialCDF = v_stripe_test('initial.nc', Lx, Ly, Nx, Ny, dt, nb_state, dX, dY)

In [6]:
mySim = Simulation(initialCDF,
                   T, Nt, 
                   methods, 
                   output_folder, 
                   save_rate, 
                   backup_rate, 
                   verbose,
                   methods_kwargs)

In [7]:
mySim.run()

          ------------------------
          |  RUNNING SIMULATION  |
          ------------------------


Iteration  0 ...
---> backup refreshed at iteration 0
---> saved results of iteration 0
      *** Proceeding to method: pseudo_spectral_wind
      *** CPU time = 0.88  seconds
      *** Proceeding to method: wrap_advection_step_3P
      *** CPU time = 5.53  seconds


Iteration  1 ...
---> saved results of iteration 1
      *** Proceeding to method: pseudo_spectral_wind
      *** CPU time = 0.76  seconds
      *** Proceeding to method: wrap_advection_step_3P
      *** CPU time = 5.55  seconds


Iteration  2 ...
---> saved results of iteration 2
      *** Proceeding to method: pseudo_spectral_wind
      *** CPU time = 0.78  seconds
      *** Proceeding to method: wrap_advection_step_3P
      *** CPU time = 5.36  seconds


Iteration  3 ...
---> saved results of iteration 3
      *** Proceeding to method: pseudo_spectral_wind
      *** CPU time = 0.74  seconds
      *** Proceeding to 

KeyboardInterrupt: 

# netCDF results can easily be analyzed...

In [8]:
from netCDF4 import Dataset
resultsCDF = Dataset(output_folder + '/results.nc', 'r', format='NETCDF4', parallel=False)
resultsCDF.__dict__

{'T': 20,
 'Lx': 2048,
 'Ly': 1024,
 'Nx': 256,
 'Ny': 128,
 'dx': 8.0,
 'dy': 8.0,
 'z_star': -500,
 'gamma_1': -0.004,
 'gamma_2': -0.0085,
 'Delta_zc': 500,
 'Delta_Tc': -5,
 'g': 9.81,
 'N_t': 0.01,
 'N_s': 0.02,
 'theta_00': 300}

In [9]:
resultsCDF['t'][:].data

array([ 0., 10.])

In [10]:
resultsCDF.close()

# <center>...and plotted !

In [ ]:
from filament.display import animate

In [ ]:
variable = 'theta_t'

# the CDF file results.nc stored in the output_folder is used to 
# generate a video saved in the output_folder
animate.make_video(output_folder + '/results.nc', output_folder, variable)

# <center>N'Joy

## To do :
- plots (both during and after computation)
- w vertical wind use in wrap_advection_3P
- WV
- check if we can start back from our backup file --> PB erase previous data in results.nc ... 

# Checking backup start

In [11]:
T = 11*dt
Nt = T//dt
from netCDF4 import Dataset
backupCDF = Dataset(output_folder + '/backup.nc', 'r', format='NETCDF4', parallel=False)

mySim = Simulation(backupCDF,
                   T, Nt, 
                   methods, 
                   output_folder, 
                   save_rate, 
                   backup_rate, 
                   verbose,
                   methods_kwargs)

In [12]:
mySim.run()

          ------------------------
          |  RUNNING SIMULATION  |
          ------------------------


Iteration  0 ...
---> backup refreshed at iteration 0
---> saved results of iteration 0
      *** Proceeding to method: pseudo_spectral_wind
      *** CPU time = 0.81  seconds
      *** Proceeding to method: wrap_advection_step_3P
      *** CPU time = 6.09  seconds


Iteration  1 ...
---> saved results of iteration 1
      *** Proceeding to method: pseudo_spectral_wind
      *** CPU time = 0.77  seconds
      *** Proceeding to method: wrap_advection_step_3P
      *** CPU time = 6.09  seconds


Iteration  2 ...
---> saved results of iteration 2
      *** Proceeding to method: pseudo_spectral_wind
      *** CPU time = 0.77  seconds
      *** Proceeding to method: wrap_advection_step_3P
      *** CPU time = 5.99  seconds


Iteration  3 ...
---> saved results of iteration 3
      *** Proceeding to method: pseudo_spectral_wind
      *** CPU time = 0.77  seconds
      *** Proceeding to 

In [15]:
resultsCDF = Dataset(output_folder + '/results.nc', 'r', format='NETCDF4', parallel=False)
print(resultsCDF['t'][:].data)
resultsCDF.close()

[100. 110. 120. 130. 140. 150. 160. 170. 180. 190. 200.]
